In [1]:
import sys
import time
import tensorflow as tf
import numpy as np
sys.path.insert(0, '../')
%load_ext autoreload
%autoreload 2

In [2]:
from seqmodel.bunch import Bunch
from seqmodel.experiment.policy_agent import PolicyAgent
from seqmodel import model
from seqmodel import data

In [3]:
vocab = data.Vocabulary.from_vocab_file('../data/tiny_copy/vocab.txt')
valid_iter = data.Seq2SeqIterator(vocab, vocab)
valid_iter.initialize('../data/tiny_copy/valid.txt')

In [4]:
tf.reset_default_graph()
agent_opt = PolicyAgent.default_opt()
emb_opt = agent_opt.policy_model.model_opt.embedding
dec_opt = agent_opt.policy_model.model_opt.decoder
enc_opt = agent_opt.policy_model.model_opt.encoder
optim_opt = agent_opt.optim

emb_opt.decoder_dim = 64
emb_opt.encoder_dim = 64

dec_opt.rnn_opt.rnn_cell.cell_opt.num_units = 64
enc_opt.rnn_opt.rnn_cell.cell_opt.num_units = 64

optim_opt.learning_rate = 0.3
optim_opt.name = 'GradientDescentOptimizer'

sess = tf.Session()
agent = PolicyAgent(agent_opt, sess)
agent.initialize_model(with_training=True)
agent.initialize_optim()
for v in tf.trainable_variables():
    print('{}, {}'.format(v.name, v.get_shape()))
sess.run(tf.global_variables_initializer())
agent.train(valid_iter, 10, valid_iter, 10, verbose=True)
info = agent.evaluate(valid_iter, 10)
print("PPL: {}, time: {}".format(
    info.eval_cost/info.num_tokens, info.end_time - info.start_time))

policy_agent/policy/model/encoder_embedding:0, (15, 64)
policy_agent/policy/model/decoder_embedding:0, (15, 64)
policy_agent/policy/model/encoder_rnn/rnn/basic_lstm_cell/weights:0, (128, 256)
policy_agent/policy/model/encoder_rnn/rnn/basic_lstm_cell/biases:0, (256,)
policy_agent/policy/model/decoder_rnn/rnn/basic_lstm_cell/weights:0, (128, 256)
policy_agent/policy/model/decoder_rnn/rnn/basic_lstm_cell/biases:0, (256,)
policy_agent/policy/model/decoder_rnn/logit_w:0, (15, 64)
policy_agent/policy/model/decoder_rnn/logit_b:0, (15,)


[INFO ]ep: 0, lr: 0.300000
[INFO ]train: @99 tr_loss: 15.22850, eval_loss: 2.28695, wps: 1058.7
[INFO ]valid: @99 tr_loss: 0.00000, eval_loss: 2.04570, wps: 2498.5
[INFO ]ep: 1, lr: 0.300000
[INFO ]train: @99 tr_loss: 12.08141, eval_loss: 1.80471, wps: 1090.5
[INFO ]valid: @99 tr_loss: 0.00000, eval_loss: 1.38181, wps: 2170.1
[INFO ]ep: 2, lr: 0.300000
[INFO ]train: @99 tr_loss: 8.32639, eval_loss: 1.24369, wps: 956.7
[INFO ]valid: @99 tr_loss: 0.00000, eval_loss: 0.98122, wps: 2220.7
[INFO ]ep: 3, lr: 0.300000
[INFO ]train: @99 tr_loss: 5.29155, eval_loss: 0.78933, wps: 1315.0
[INFO ]valid: @99 tr_loss: 0.00000, eval_loss: 0.53771, wps: 2631.0
[INFO ]ep: 4, lr: 0.300000
[INFO ]train: @99 tr_loss: 3.60067, eval_loss: 0.53304, wps: 1151.0
[INFO ]valid: @99 tr_loss: 0.00000, eval_loss: 0.49340, wps: 4286.7
[INFO ]ep: 5, lr: 0.300000
[INFO ]train: @99 tr_loss: 2.65915, eval_loss: 0.39169, wps: 1188.7
[INFO ]valid: @99 tr_loss: 0.00000, eval_loss: 0.29793, wps: 2356.5
[INFO ]ep: 6, lr: 0.3

PPL: 0.117503514885, time: 2.15334415436


In [5]:
test_data = ([['d e f', 'd e f'], ['a b c d', 'a b c d']])
test_iter = data.Seq2SeqIterator(vocab, vocab)
test_iter.initialize(test_data)
test_iter.init_batch(2)
env = data.env.CopyEnv(test_iter, re_init=False, reward_mode=data.env.ToyRewardMode.EACH_MATCH)
transitions, packed_t, packed_r = agent.rollout(env, greedy=True)

In [6]:
test_iter.init_batch(2)
env = data.env.CopyEnv(test_iter, re_init=False, reward_mode=data.env.ToyRewardMode.EACH_MATCH)
agent.evaluate_policy(env)

(<seqmodel.experiment.run_info.RunningInfo at 0x7f62a69902d0>, 1.0)

In [7]:
test_iter.init_batch(2)
env = data.env.CopyEnv(test_iter, re_init=False, reward_mode=data.env.ToyRewardMode.EACH_MATCH)
_ = agent.policy_gradient(env, 2, env)

1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
